In [13]:
import torch, nnunetv2
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("nnU-Net v2 imported successfully!")

# Test that you can access nnunetv2 modules
print("nnunetv2 location:", nnunetv2.__file__)

Torch: 2.5.1+cu121
CUDA available: True
nnU-Net v2 imported successfully!
nnunetv2 location: C:\Users\User\anaconda3\envs\nnunet_new\Lib\site-packages\nnunetv2\__init__.py


In [14]:
import os, nnunetv2, torch

# Work directories - updated for your local setup
BASE = "D:/nnunet_with_classification/data"
RAW  = f"{BASE}/nnUNet_raw"
PREP = f"{BASE}/nnUNet_preprocessed"
RES  = f"{BASE}/nnUNet_results"

# Create nnU-Net directories
os.makedirs(RAW, exist_ok=True)
os.makedirs(PREP, exist_ok=True)
os.makedirs(RES, exist_ok=True)

# Set environment variables
os.environ["nnUNet_raw"] = RAW
os.environ["nnUNet_preprocessed"] = PREP
os.environ["nnUNet_results"] = RES

print("Torch:", torch.__version__, "| CUDA:", torch.version.cuda)
print("nnU-Net v2: imported successfully!")
print(f"nnU-Net directories created at: {BASE}")

Torch: 2.5.1+cu121 | CUDA: 12.1
nnU-Net v2: imported successfully!
nnU-Net directories created at: D:/nnunet_with_classification/data


In [ ]:
import os, glob, pathlib, shutil, re, json, csv

# >>>>>>>>>>>> UPDATE THIS PATH <<<<<<<<<<<<
SRC = "D:/nnunet_with_classification/data"

DSID = 777
DSNAME = f"Dataset{DSID:03d}_M31Quiz"
RAW  = os.environ["nnUNet_raw"]
PREP = os.environ["nnUNet_preprocessed"]
DSROOT = f"{RAW}/{DSNAME}"
imgTr = f"{DSROOT}/imagesTr"; lblTr = f"{DSROOT}/labelsTr"; imgTs = f"{DSROOT}/imagesTs"
for d in (imgTr, lblTr, imgTs): os.makedirs(d, exist_ok=True)

def stem(p): return pathlib.Path(p).name.replace(".nii.gz","")
sub_regex = re.compile(r"subtype\s*([012])", re.IGNORECASE)

def find_split(name):
    # case-insensitive split folder lookup
    for cand in os.listdir(SRC):
        if cand.lower() == name:
            p = os.path.join(SRC, cand)
            if os.path.isdir(p): return p
    return None

train_dir = find_split("train")
val_dir   = find_split("validation")
test_dir  = find_split("test")
assert train_dir and val_dir and test_dir, f"Could not find train/validation/test under {SRC}"

def ingest_split(split_dir, cls_map):
    imgs = glob.glob(os.path.join(split_dir, "**", "*_0000.nii.gz"), recursive=True)
    used = 0
    for img in sorted(imgs):
        case = stem(img).replace("_0000","")
        msk  = img.replace("_0000.nii.gz",".nii.gz")
        if not os.path.exists(msk):
            continue
        shutil.copy(img, f"{imgTr}/{case}_0000.nii.gz")
        shutil.copy(msk, f"{lblTr}/{case}.nii.gz")
        used += 1
        # infer subtype from folder names (expects 'subtype0/1/2')
        sub_idx = None
        for part in pathlib.Path(img).parts:
            m = sub_regex.search(part)
            if m:
                sub_idx = int(m.group(1)); break
        if sub_idx is not None:
            cls_map[case] = sub_idx
    return used

cls_map = {}
n_tr = ingest_split(train_dir, cls_map)
n_va = ingest_split(val_dir,   cls_map)

# test images
for img in sorted(glob.glob(os.path.join(test_dir, "**", "*_0000.nii.gz"), recursive=True)):
    case = stem(img).replace("_0000","")
    shutil.copy(img, f"{imgTs}/{case}_0000.nii.gz")

print("imagesTr:", len(glob.glob(f"{imgTr}/*_0000.nii.gz")))
print("labelsTr:", len(glob.glob(f"{lblTr}/*.nii.gz")))
print("imagesTs:", len(glob.glob(f"{imgTs}/*_0000.nii.gz")))
print("Mapped classification labels:", len(cls_map))

# dataset.json
dataset_json = {
  "name": DSNAME,
  "tensorImageSize": "3D",
  "modality": {"0": "CT"},
  "labels": {"background": 0, "pancreas": 1, "lesion": 2},
  "numTraining": len(glob.glob(f"{lblTr}/*.nii.gz")),
  "numTest": len(glob.glob(f"{imgTs}/*_0000.nii.gz")),
  "training": [{"image": f"./imagesTr/{stem(i)}.nii.gz",
                "label": f"./labelsTr/{stem(i).replace('_0000','')}.nii.gz"}
               for i in sorted(glob.glob(f"{imgTr}/*_0000.nii.gz"))],
  "test": [f"./imagesTs/{stem(i)}.nii.gz" for i in sorted(glob.glob(f"{imgTs}/*_0000.nii.gz"))]
}
os.makedirs(DSROOT, exist_ok=True)
with open(f"{DSROOT}/dataset.json","w") as f: json.dump(dataset_json, f, indent=2)

# splits_final.json (use original validation as nnU-Net val)
val_cases = {stem(p).replace("_0000","") for p in glob.glob(os.path.join(val_dir, "**", "*_0000.nii.gz"), recursive=True)}
all_cases = sorted([stem(p).replace("_0000","") for p in glob.glob(f"{imgTr}/*_0000.nii.gz")])
train_cases = [c for c in all_cases if c not in val_cases]
spdir = f"{PREP}/{DSNAME}"
os.makedirs(spdir, exist_ok=True)
with open(f"{spdir}/splits_final.json","w") as f:
    json.dump([{"train": train_cases, "val": sorted(list(val_cases))}], f, indent=2)

# classification_labels.csv
csv_path = f"{spdir}/classification_labels.csv"
with open(csv_path, "w", newline="") as f:
    w = csv.writer(f)
    for c in all_cases:
        if c in cls_map:
            w.writerow([c, cls_map[c]])
        else:
            # if you see many misses here, your folders may not be named 'subtype0/1/2'
            pass

print("Wrote dataset.json, splits_final.json, classification_labels.csv")

In [ ]:
import json, glob, os, pathlib

DSID = 777
DSNAME = f"Dataset{DSID:03d}_M31Quiz"
RAW   = os.environ["nnUNet_raw"]
DSROOT = f"{RAW}/{DSNAME}"
lblTr = f"{DSROOT}/labelsTr"

dataset_v2 = {
    "channel_names": { "0": "CT" },               # required in v2
    "labels": { "background": 0, "pancreas": 1, "lesion": 2 },  # your classes
    "numTraining": len(glob.glob(os.path.join(lblTr, "*.nii.gz"))),
    "file_ending": ".nii.gz"
}
with open(f"{DSROOT}/dataset.json", "w") as f:
    json.dump(dataset_v2, f, indent=2)

print("✅ Wrote v2 dataset.json at:", f"{DSROOT}/dataset.json")
print("numTraining:", dataset_v2["numTraining"])

In [ ]:
import nibabel as nib, numpy as np, glob, os, pathlib

DSID = 777
DSNAME = f"Dataset{DSID:03d}_M31Quiz"
lblTr = os.path.join(os.environ["nnUNet_raw"], DSNAME, "labelsTr")

bad = []
for p in sorted(glob.glob(os.path.join(lblTr, "*.nii.gz"))):
    img = nib.load(p)
    arr = img.get_fdata()  # floats possible
    uniq = np.unique(arr)
    if not np.all(np.isin(uniq, [0,1,2])):
        bad.append((p, uniq))

print("Masks needing fix:", len(bad))
for p, uniq in bad[:10]:
    print("  ", pathlib.Path(p).name, "unique:", uniq[:10])

# Fix: round and clip to {0,1,2}, write as uint8
for p, uniq in bad:
    img = nib.load(p)
    arr = img.get_fdata()
    arr = np.rint(arr)           # round to nearest integer
    arr = np.clip(arr, 0, 2)     # enforce label set
    arr = arr.astype(np.uint8)

    hdr = img.header.copy()
    hdr.set_data_dtype(np.uint8)
    # avoid unintended scaling
    hdr["scl_slope"] = 1
    hdr["scl_inter"] = 0

    fixed = nib.Nifti1Image(arr, img.affine, hdr)
    nib.save(fixed, p)

# Sanity check after fix
remaining = []
for p in sorted(glob.glob(os.path.join(lblTr, "*.nii.gz"))):
    arr = nib.load(p).get_fdata()
    uniq = np.unique(arr)
    if not np.all(np.isin(uniq, [0,1,2])):
        remaining.append((p, uniq))
print("Remaining masks with non-{0,1,2} labels:", len(remaining))

In [ ]:
!nnUNetv2_plan_and_preprocess -d 777 -c 3d_fullres --verify_dataset_integrity -pl nnUNetPlannerResEncM

In [ ]:
!nnUNetv2_train 777 3d_fullres 0 -tr nnUNetTrainerWithClassification -p nnUNetResEncUNetMPlans --c

In [ ]:
# to get inference on validation for segmentation

!python scripts/inference.py \
  --model_dir "D:/nnunet_with_classification/data/nnUNet_results/Dataset777_M31Quiz/nnUNetTrainerWithClassification__nnUNetResEncUNetMPlans__3d_fullres" \
  --input_dir "D:/nnunet_with_classification/data/validation_flat" \
  --output_dir "D:/nnunet_with_classification/data/predication_seg_final_best" \
  --fold 0 \
  --checkpoint checkpoint_best.pth \
  --num_classes 3 \
  --no-tta \
  --preproc_workers 1 \
  --export_workers 1

In [ ]:
# to get inference on validation for classifictaion

!python scripts/inference_probs_both.py \
  --model_dir "D:/nnunet_with_classification/data/nnUNet_results/Dataset777_M31Quiz/nnUNetTrainerWithClassification__nnUNetResEncUNetMPlans__3d_fullres" \
  --input_dir "D:/nnunet_with_classification/data/validation_flat" \
  --output_dir "D:/nnunet_with_classification/data/predication_classification_final_both_probs" \
  --fold 0 \
  --checkpoint checkpoint_best_classification.pth \
  --num_classes 3 \
  --no-tta \
  --preproc_workers 1 \
  --export_workers 1

In [ ]:
import sys
sys.path.append(r'D:\nnunet_with_classification\scripts')
import balanced_tile_to_csv

In [4]:
jsonl_file = r"D:\nnunet_with_classification\data\predication_classification_final_both_probs\classification_tile_probs.jsonl"
output_file = r"D:\nnunet_with_classification\data\predication_classification_final_both_probs\balanced_predictions.csv"
# This gives equal weight to each predicted class, not each tile
df = balanced_tile_to_csv.convert_tiles_balanced(
    jsonl_file, 
    output_file, 
    method='class_balanced'
)
df.head()

Converting D:\nnunet_with_classification\data\predication_classification_final_both_probs\classification_tile_probs.jsonl using class_balanced method
Saved 36 predictions to D:\nnunet_with_classification\data\predication_classification_final_both_probs\balanced_predictions.csv
Class distribution:
   Class 0: 7 cases (19.4%) - Avg conf: 0.498
   Class 1: 27 cases (75.0%) - Avg conf: 0.413
   Class 2: 2 cases (5.6%) - Avg conf: 0.426


,case,pred_class,Confidence,Method,Prob_Class_0,Prob_Class_1,Prob_Class_2
0,quiz_0_168.nii.gz,1,0.4182,class_balanced,0.2742,0.4182,0.3076
1,quiz_0_171.nii.gz,1,0.3988,class_balanced,0.3565,0.3988,0.2446
2,quiz_0_174.nii.gz,0,0.6572,class_balanced,0.6572,0.2272,0.1156
3,quiz_0_184.nii.gz,1,0.4284,class_balanced,0.2385,0.4284,0.3330
4,quiz_0_187.nii.gz,1,0.4377,class_balanced,0.3217,0.4377,0.2406


In [5]:
# Compare with other methods
results = balanced_tile_to_csv.compare_methods(
    jsonl_file, 
    r"D:\nnunet_with_classification\data\predication_classification_final_both_probs\method_comparison"
)


--- Testing class_balanced ---
Converting D:\nnunet_with_classification\data\predication_classification_final_both_probs\classification_tile_probs.jsonl using class_balanced method
Saved 36 predictions to D:\nnunet_with_classification\data\predication_classification_final_both_probs\method_comparison\predictions_class_balanced.csv
Class distribution:
   Class 0: 7 cases (19.4%) - Avg conf: 0.498
   Class 1: 27 cases (75.0%) - Avg conf: 0.413
   Class 2: 2 cases (5.6%) - Avg conf: 0.426

--- Testing minority_boost ---
Converting D:\nnunet_with_classification\data\predication_classification_final_both_probs\classification_tile_probs.jsonl using minority_boost method
Saved 36 predictions to D:\nnunet_with_classification\data\predication_classification_final_both_probs\method_comparison\predictions_minority_boost.csv
Class distribution:
   Class 0: 7 cases (19.4%) - Avg conf: 0.572
   Class 1: 26 cases (72.2%) - Avg conf: 0.513
   Class 2: 3 cases (8.3%) - Avg conf: 0.514

--- Testing ent

In [12]:
# Fixed evaluation script - handles .nii.gz extension in CSV
import os
import csv
import numpy as np
import nibabel as nib
from sklearn.metrics import f1_score, classification_report, precision_recall_fscore_support, confusion_matrix

# Validation cases (base names without extension)
validation_cases = [
    "quiz_0_168", "quiz_0_171", "quiz_0_174", "quiz_0_184", "quiz_0_187", 
    "quiz_0_189", "quiz_0_244", "quiz_0_253", "quiz_0_254", "quiz_1_090",
    "quiz_1_093", "quiz_1_094", "quiz_1_154", "quiz_1_158", "quiz_1_164",
    "quiz_1_166", "quiz_1_211", "quiz_1_213", "quiz_1_221", "quiz_1_227",
    "quiz_1_231", "quiz_1_242", "quiz_1_331", "quiz_1_335", "quiz_2_074",
    "quiz_2_080", "quiz_2_084", "quiz_2_085", "quiz_2_088", "quiz_2_089",
    "quiz_2_098", "quiz_2_191", "quiz_2_241", "quiz_2_364", "quiz_2_377",
    "quiz_2_379"
]

# Paths
gt_seg_dir   = f"{os.environ['nnUNet_raw']}/Dataset777_M31Quiz/labelsTr"
pred_seg_dir = "D:/nnunet_with_classification/data/predication_seg_final_best"
gt_cls_csv   = "D:/nnunet_with_classification/data/validation_labels.csv"
pred_cls_csv = "D:/nnunet_with_classification/data/predication_classification_final_both_probs/method_comparison/predictions_class_balanced.csv"

def clean_case_name(case_name):
    """Remove .nii.gz extension and strip whitespace"""
    return str(case_name).strip().replace('.nii.gz', '')

def dice_score(y_true, y_pred):
    """Compute Dice Similarity Coefficient"""
    intersection = np.sum(y_true * y_pred)
    total = np.sum(y_true) + np.sum(y_pred)
    return (2.0 * intersection) / total if total > 0 else 1.0

# Check paths
print("🔍 Checking paths...")
print(f"GT segmentation dir:          {gt_seg_dir}   -> {os.path.exists(gt_seg_dir)}")
print(f"Predicted segmentation dir:   {pred_seg_dir} -> {os.path.exists(pred_seg_dir)}")
print(f"GT classification CSV:        {gt_cls_csv}   -> {os.path.exists(gt_cls_csv)}")
print(f"Predicted classification CSV: {pred_cls_csv} -> {os.path.exists(pred_cls_csv)}")

# Segmentation metrics
print("\n📊 Computing segmentation metrics...")
pancreas_dices, lesion_dices, missing_files = [], [], []

for case in validation_cases:
    gt_file   = f"{gt_seg_dir}/{case}.nii.gz"
    pred_file = f"{pred_seg_dir}/{case}.nii.gz"
    
    if os.path.exists(gt_file) and os.path.exists(pred_file):
        try:
            gt   = nib.load(gt_file).get_fdata()
            pred = nib.load(pred_file).get_fdata()
            
            # Whole pancreas (>0)
            gt_pancreas   = (gt  > 0).astype(np.uint8)
            pred_pancreas = (pred > 0).astype(np.uint8)
            pancreas_dices.append(dice_score(gt_pancreas, pred_pancreas))
            
            # Lesion (== 2)
            gt_lesion   = (gt  == 2).astype(np.uint8)
            pred_lesion = (pred == 2).astype(np.uint8)
            lesion_dices.append(dice_score(gt_lesion, pred_lesion))
        except Exception as e:
            print(f"Error processing {case}: {e}")
            missing_files.append(case)
    else:
        missing_files.append(case)

if pancreas_dices:
    whole_pancreas_dsc = float(np.mean(pancreas_dices))
    lesion_dsc         = float(np.mean(lesion_dices))
    print(f"Pancreas DSC (mean): {whole_pancreas_dsc:.4f}")
    print(f"Lesion   DSC (mean): {lesion_dsc:.4f}")
else:
    whole_pancreas_dsc = 0.0
    lesion_dsc         = 0.0
    print("No matched segmentation files found.")

# Classification metrics - FIXED to handle .nii.gz extensions
print("\n🎯 Computing classification metrics...")

# Load GT labels (handle different column names)
gt_labels = {}
if os.path.exists(gt_cls_csv):
    with open(gt_cls_csv, 'r', newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        
        # Try different possible column names
        case_col = None
        label_col = None
        
        for col in ['case', 'Cases', 'Names', 'Case', 'names']:
            if col in reader.fieldnames:
                case_col = col
                break
        
        for col in ['label', 'Label', 'Subtype', 'subtype', 'pred_class']:
            if col in reader.fieldnames:
                label_col = col
                break
        
        if not case_col:
            case_col = reader.fieldnames[0]  # Use first column
        if not label_col:
            label_col = reader.fieldnames[1]  # Use second column
        
        print(f"Using GT columns: case='{case_col}', label='{label_col}'")
        
        # Reset file pointer
        f.seek(0)
        reader = csv.DictReader(f)
        
        for row in reader:
            case_name = clean_case_name(row[case_col])
            gt_labels[case_name] = int(row[label_col])

# Load prediction labels
pred_labels = {}
if os.path.exists(pred_cls_csv):
    with open(pred_cls_csv, 'r', newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        
        # Try different possible column names
        case_col = None
        for col in ['case', 'Cases', 'Names', 'Case', 'names']:
            if col in reader.fieldnames:
                case_col = col
                break
        
        if not case_col:
            case_col = reader.fieldnames[0]  # Use first column
        
        print(f"Using prediction case column: '{case_col}'")
        
        # Check for pred_class column or probability columns
        has_pred_class = 'pred_class' in reader.fieldnames
        prob_cols = [c for c in reader.fieldnames if c.startswith('Prob_Class_')]
        
        for row in reader:
            case_name = clean_case_name(row[case_col])
            
            if has_pred_class and row.get('pred_class', '') != '':
                pred_labels[case_name] = int(row['pred_class'])
            elif prob_cols:
                probs = np.array([float(row[p]) for p in prob_cols], dtype=float)
                pred_labels[case_name] = int(np.argmax(probs))

print(f"\nFound {len(gt_labels)} GT labels and {len(pred_labels)} predictions")

# Match validation cases with GT and predictions
val_gt, val_pred = [], []
matched_cases = []

for case in validation_cases:
    if case in gt_labels and case in pred_labels:
        val_gt.append(gt_labels[case])
        val_pred.append(pred_labels[case])
        matched_cases.append(case)

print(f"Matched {len(matched_cases)} cases for evaluation")

if val_gt and val_pred:
    # Overall metrics
    macro_f1 = f1_score(val_gt, val_pred, average='macro')
    micro_f1 = f1_score(val_gt, val_pred, average='micro')
    weighted_f1 = f1_score(val_gt, val_pred, average='weighted')
    accuracy = np.mean(np.array(val_gt) == np.array(val_pred))
    
    print(f"\nOverall Accuracy: {accuracy:.4f}")
    print(f"Macro F1:         {macro_f1:.4f}")
    print(f"Micro F1:         {micro_f1:.4f}")
    print(f"Weighted F1:      {weighted_f1:.4f}")
    
    # Per-class metrics
    precision, recall, f1, support = precision_recall_fscore_support(val_gt, val_pred, average=None, zero_division=0)
    
    print("\n📈 Per-Class Classification Metrics:")
    print("-" * 60)
    print(f"{'Class':<10} {'Precision':<12} {'Recall':<10} {'F1-Score':<10} {'Support':<8}")
    print("-" * 60)
    
    class_names = ['Subtype 0', 'Subtype 1', 'Subtype 2']
    for i, class_name in enumerate(class_names):
        if i < len(precision):
            print(f"{class_name:<10} {precision[i]:<12.4f} {recall[i]:<10.4f} {f1[i]:<10.4f} {support[i]:<8}")
        else:
            print(f"{class_name:<10} {'0.0000':<12} {'0.0000':<10} {'0.0000':<10} {'0':<8}")
    
    # Confusion Matrix
    cm = confusion_matrix(val_gt, val_pred)
    print("\n🎯 Confusion Matrix:")
    print("-" * 40)
    print("Predicted ->", end="")
    for i in range(3):
        print(f"  {i:>3}", end="")
    print()
    print("True ↓")
    for i in range(3):
        print(f"    {i:>3}     ", end="")
        for j in range(3):
            if i < cm.shape[0] and j < cm.shape[1]:
                print(f"  {cm[i,j]:>3}", end="")
            else:
                print(f"  {0:>3}", end="")
        print()
    
else:
    macro_f1 = 0.0
    accuracy = 0.0
    print("No overlapping classification cases found between GT and predictions.")

# Final summary
print("\n" + "="*50)
print("                    SUMMARY")
print("="*50)
print(f"Whole Pancreas DSC: {whole_pancreas_dsc:.4f}")
print(f"Lesion DSC:         {lesion_dsc:.4f}")
print(f"Classification Acc: {accuracy:.4f}")
print(f"Macro F1:           {macro_f1:.4f}")
print("="*50)

if missing_files:
    print(f"\n⚠️  Missing seg files for {len(missing_files)} cases")

🔍 Checking paths...
GT segmentation dir:          D:/nnunet_with_classification/data/nnUNet_raw/Dataset777_M31Quiz/labelsTr   -> True
Predicted segmentation dir:   D:/nnunet_with_classification/data/predication_seg_final_best -> True
GT classification CSV:        D:/nnunet_with_classification/data/validation_labels.csv   -> True
Predicted classification CSV: D:/nnunet_with_classification/data/predication_classification_final_both_probs/method_comparison/predictions_class_balanced.csv -> True

📊 Computing segmentation metrics...
Pancreas DSC (mean): 0.9029
Lesion   DSC (mean): 0.5811

🎯 Computing classification metrics...
Using GT columns: case='case', label='label'
Using prediction case column: 'case'

Found 288 GT labels and 36 predictions
Matched 36 cases for evaluation

Overall Accuracy: 0.5278
Macro F1:         0.4425
Micro F1:         0.5278
Weighted F1:      0.4668

📈 Per-Class Classification Metrics:
------------------------------------------------------------
Class      Precisi

In [ ]:
# to get segmentation inference on test data

!python scripts/inference.py \
  --model_dir "D:/nnunet_with_classification/data/nnUNet_results/Dataset777_M31Quiz/nnUNetTrainerWithClassification__nnUNetResEncUNetMPlans__3d_fullres" \
  --input_dir "D:/nnunet_with_classification/data/nnUNet_raw/Dataset777_M31Quiz/imagesTs" \
  --output_dir "D:/nnunet_with_classification/data/new_test_label_predictions_segmentation_best" \
  --fold 0 \
  --checkpoint checkpoint_best.pth \
  --num_classes 3 \
  --no-tta \
  --preproc_workers 1 \
  --export_workers 1

In [ ]:
# to get classification inference on test data


!python scripts/inference_probs_both.py \
  --model_dir "D:/nnunet_with_classification/data/nnUNet_results/Dataset777_M31Quiz/nnUNetTrainerWithClassification__nnUNetResEncUNetMPlans__3d_fullres" \
  --input_dir "D:/nnunet_with_classification/data/nnUNet_raw/Dataset777_M31Quiz/imagesTs" \
  --output_dir "D:/nnunet_with_classification/data/predication_classification_final_both_probs_test_data" \
  --fold 0 \
  --checkpoint checkpoint_best_classification.pth \
  --num_classes 3 \
  --no-tta \
  --preproc_workers 1 \
  --export_workers 1

In [16]:
import sys
sys.path.append(r'D:\nnunet_with_classification\scripts')
import balanced_tile_to_csv

In [18]:
jsonl_file = r"D:\nnunet_with_classification\data\predication_classification_final_both_probs_test_data\classification_tile_probs.jsonl"
output_file = r"D:\nnunet_with_classification\data\predication_classification_final_both_probs_test_data\balanced_predictions.csv"
# This gives equal weight to each predicted class, not each tile
df = balanced_tile_to_csv.convert_tiles_balanced(
    jsonl_file, 
    output_file, 
    method='class_balanced'
)
df.head()

Converting D:\nnunet_with_classification\data\predication_classification_final_both_probs_test_data\classification_tile_probs.jsonl using class_balanced method
Saved 72 predictions to D:\nnunet_with_classification\data\predication_classification_final_both_probs_test_data\balanced_predictions.csv
Class distribution:
   Class 0: 10 cases (13.9%) - Avg conf: 0.444
   Class 1: 60 cases (83.3%) - Avg conf: 0.409
   Class 2: 2 cases (2.8%) - Avg conf: 0.378


,case,pred_class,Confidence,Method,Prob_Class_0,Prob_Class_1,Prob_Class_2
0,quiz_037.nii.gz,1,0.4145,class_balanced,0.2735,0.4145,0.3120
1,quiz_045.nii.gz,1,0.4261,class_balanced,0.3623,0.4261,0.2116
2,quiz_047.nii.gz,1,0.3917,class_balanced,0.3011,0.3917,0.3072
3,quiz_048.nii.gz,1,0.4071,class_balanced,0.2649,0.4071,0.3280
4,quiz_052.nii.gz,1,0.4028,class_balanced,0.2672,0.4028,0.3300
